In [3]:
import polars as pl
import numpy as np
import os
import matplotlib.pyplot as plt
import emd
from dtw import *
from utils import *

time = []
fs = []
hr = []
mx = []

# Recordings files paths
path = os.getcwd() + r"\data_healthy"
file_paths = sorted([os.path.join(path, file) for file in os.listdir(path)], key=extract_recording_number)

# Read data from the files
for file_path in file_paths:
    df = pl.read_csv(file_path, separator=";", decimal_comma=True)
    t_hat, _ = convert_datetime_to_time(df["DateTime"].to_numpy())
    time.append(t_hat)
    hr.append(df["hr"].to_numpy())
    if df["FVL"].mean() > df["FVR"].mean():
        mx.append(df["mx_l"].to_numpy())
    else:
        mx.append(df["mx_r"].to_numpy())

In [4]:
dtw_distances = []
dtw_normalized_distances = []
drop_dtw_distances = []
drop_dtw_normalized_distances = []

for record_number in range(len(mx)):
    imf_mx = emd.sift.sift(mx[record_number])
    n = imf_mx.shape[1]
    imf_mx = imf_mx[:, n - 3] + imf_mx[:, n - 2] + imf_mx[:, n - 1]

    imf_hr = emd.sift.sift(hr[record_number])
    n = imf_hr.shape[1]
    imf_hr = np.sum(imf_hr[:, 1:], axis=1)

    imf_hr = (imf_hr - np.min(imf_hr)) / (np.max(imf_hr) - np.min(imf_hr))
    imf_mx = (imf_mx - np.min(imf_mx)) / (np.max(imf_mx) - np.min(imf_mx))
    
    # DTW
    alignment = dtw(imf_hr, imf_mx, keep_internals=False, distance_only=False)
    dtw_distances.append(alignment.distance)
    normalized_distance = alignment.distance / (len(imf_hr) + len(imf_mx))
    dtw_normalized_distances.append(normalized_distance)

    # DropDTW
    zx_costs, x_drop_costs, z_drop_costs = compute_all_costs(
        series1=imf_hr, 
        series2=imf_mx, 
        drop_cost_type="percentile", 
        percentile=75
    )

    min_cost, matched_indices, dropped1, dropped2 = double_drop_dtw(
        costs=zx_costs,
        drop_costs1=x_drop_costs,
        drop_costs2=z_drop_costs,
        contiguous=True
    )

    drop_dtw_distances.append(min_cost)
    normalized_drop_distance = min_cost / (len(imf_hr) + len(imf_mx))
    drop_dtw_normalized_distances.append(normalized_drop_distance)

print("DTW Distances:", dtw_distances)
print("DTW Normalized Distances:", dtw_normalized_distances)
print("DropDTW Distances:", drop_dtw_distances)
print("DropDTW Normalized Distances:", drop_dtw_normalized_distances)

DTW Distances: [31.514789344786376, 37.85929151709315, 90.1951290778901, 43.64662106135333, 27.18584108157392, 27.512710932348693, 31.951457300149823, 24.812040385207588, 35.99447106456082, 38.66905337001282, 50.929339670775335, 101.1494521719999, 42.379021504395865, 59.390039878161325, 60.89384208642561, 13.118137731539457, 21.533677518627137, 39.668641007302774, 15.173883673106058, 57.22080627210443, 47.43218527398428, 45.34764811688677, 50.03818387962093, 44.22849760956807, 30.740514392263613, 36.11517161192916, 37.901767665536184, 52.81985265541882, 37.38937316525793, 35.584309736314985, 39.118978480013475, 32.477419550829545, 13.358599229908469, 42.74538906768685]
DTW Normalized Distances: [0.08249944854656119, 0.0673652873969629, 0.15658876576022587, 0.10697701240527777, 0.05988070722813638, 0.0674331150302664, 0.06028576849084872, 0.08555875994899169, 0.0705773942442369, 0.06760324015736506, 0.08516611985079488, 0.2022989043439998, 0.068574468453715, 0.12120416301665576, 0.09574

In [16]:
for record_number in range(5):
    
    
    imf_mx = emd.sift.sift(mx[record_number])
    n = imf_mx.shape[1]
    imf_sum_mx = imf_mx[:, n - 3] + imf_mx[:, n - 2] + imf_mx[:, n - 1]

    imf_hr = emd.sift.sift(hr[record_number])
    n = imf_hr.shape[1]
    imf_sum_hr = imf_hr[:, n - 3] + imf_hr[:, n - 2] + imf_hr[:, n - 1]

    # Min-max normalize imf_sum_hr and imf_sum_mx
    imf_sum_hr = (imf_sum_hr - np.min(imf_sum_hr)) / (np.max(imf_sum_hr) - np.min(imf_sum_hr))
    imf_sum_mx = (imf_sum_mx - np.min(imf_sum_mx)) / (np.max(imf_sum_mx) - np.min(imf_sum_mx))
    
    # DropDTW
    zx_costs, x_drop_costs, z_drop_costs = compute_all_costs(
        series1=imf_sum_hr, 
        series2=imf_sum_mx, 
        drop_cost_type="percentile", 
        percentile=70
    )

    min_cost, matched_indices, dropped1, dropped2 = double_drop_dtw(
        costs=zx_costs,
        drop_costs1=x_drop_costs,
        drop_costs2=z_drop_costs,
        contiguous=True
    )

    print(min_cost)



15658.137611852886
19155.718817788576


KeyboardInterrupt: 

In [15]:
import pandas as pd

# Create a DataFrame with the distances
distances_df = pd.DataFrame({
    'DTW Distances': dtw_distances,
    'DTW Normalized Distances': dtw_normalized_distances,
    'DropDTW Distances': drop_dtw_distances,
    'DropDTW Normalized Distances': drop_dtw_normalized_distances
})

# Save the DataFrame to a CSV file
distances_path = r'E:\_PracaDyplomowa\Analysis'
distances_df.to_csv(os.path.join(distances_path, 'distances.csv'), index=False)